In [1]:
import pandas as pd

In [2]:
parallel_corpora = pd.read_csv('parallel_corpora.csv')

In [3]:
def create_dictionary(language1: list[list[str]], language2: list[list[str]]):
    mapping_counts = {}
    for (index, (sentence1, sentence2)) in enumerate(zip(language1, language2)):
        for word1 in sentence1:
            for word2 in sentence2:
                if (word1, word2) not in mapping_counts:
                    mapping_counts[(word1, word2)] = 1
                mapping_counts[(word1, word2)] = mapping_counts[(word1, word2)] + 1

    return mapping_counts

In [4]:
def create_sentences_df(parallel_corpora: pd.DataFrame, language1: str, language2: str, convert_lowercase: bool | None):
    convert_lowercase = False if convert_lowercase is None else convert_lowercase
    pairs = parallel_corpora.loc[
        (parallel_corpora['language1'] == language1) &
        (parallel_corpora['language2'] == language2)
    ][['language1_text', 'language2_text']]

    convert_function = (lambda x: str(x)
                                 .lower()
                                 .replace(',', ' ')
                                 .replace('.', ' ')
                                 .replace('\'', ' ')
                                 .replace('-', ' ')
                                 .replace('"', ' ')
                                 .replace('!', ' ')
                                 .replace(';', ' ')
                                 .replace('-', ' ')
                                 .replace('&', ' ')
                                 .replace('$', ' ')
                                 .replace('?', ' ')
                                 .replace('(', ' ')
                                 .replace(')', ' ')
                                 .replace('[', ' ')
                                 .replace(']', ' ')
                                 .replace('*', ' ')
                                 .replace('«', ' ')
                                 .replace('»', ' ')
                                 .replace('¡', ' ')
                                 .replace('¿', ' ')
                                 .replace('/', ' ')
                                 .replace('~', ' ')
                                 .replace('=', ' ')
                                 .replace('#', '')
                                 .replace('0', '')
                                 .replace('1', '')
                                 .replace('2', '')
                                 .replace('3', '')
                                 .replace('4', '')
                                 .replace('5', '')
                                 .replace('6', '')
                                 .replace('7', '')
                                 .replace('8', '')
                                 .replace('9', '')
                                 .replace(':', '')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('\r\n', '\n')
                                 .replace('\n', ' ')
                                 .strip()
                                 .split(' ')) if convert_lowercase else (lambda x: str(x).strip().split(' '))
    
    pairs['language1_text'] = pairs['language1_text'].apply(convert_function).apply(lambda x: [word for word in x if word != ''])
    pairs['language2_text'] = pairs['language2_text'].apply(convert_function).apply(lambda x: [word for word in x if word != ''])
    return pairs

In [5]:
def create_sentences(df: pd.DataFrame):
    return (df['language1_text'].tolist(), df['language2_text'].tolist())

In [6]:
df = create_sentences_df(parallel_corpora, 'english', 'filipino', True)
(english_sentences, filipino_sentences) = create_sentences(df)
df.head(5)

,language1_text,language2_text
965446,"[in, the, beginning, god, created, the, heaven...","[nang, pasimula, nilikha, ng, diyos, ang, lang..."
965447,"[and, the, earth, was, without, form, and, voi...","[ang, lupa, ay, walang, anyo, at, walang, lama..."
965448,"[and, god, said, let, there, be, light, and, t...","[at, sinabi, ng, diyos, magkaroon, ng, liwanag..."
965449,"[and, god, saw, the, light, that, it, was, goo...","[nakita, ng, diyos, na, ang, liwanag, ay, mabu..."
965450,"[and, god, called, the, light, day, and, the, ...","[tinawag, ng, diyos, ang, liwanag, na, araw, a..."


In [7]:
dictionary = create_dictionary(english_sentences, filipino_sentences)


In [8]:
def dictionary_to_df(dictionary: dict[tuple[str, str], int]) -> pd.DataFrame:
    rows = []
    for ((word1, word2), count) in dictionary.items():
        rows.append((word1, word2, count))
    return pd.DataFrame(data=rows, columns=['word1', 'word2', 'count'])

In [9]:
dictionary_df = dictionary_to_df(dictionary)
dictionary_df.head(5)

,word1,word2,count
0,in,nang,2104
1,in,pasimula,53
2,in,nilikha,9
3,in,ng,24100
4,in,diyos,1956


In [10]:
# https://stackoverflow.com/questions/43899886/how-do-i-just-keep-the-rows-with-the-maximum-value-in-a-column-for-items-of-the
